In [1]:
!rm -rf /root/.cache/

***INSTALLING DEPENDENCIES***

In [2]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip install -q datasets loralib sentencepiece 
!pip -q install bitsandbytes accelerate
!pip -q install langchain
!pip install gradio
!pip install pyngrok
!pip install fastapi
!pip install nest-asyncio
!pip install uvicorn
!pip install requests
!pip install pypi-json
!pip install azure-cognitiveservices-speech
!pip install xformers
!pip install sentence_transformers
!pip install faiss-gpu

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31

***IMPORTING MODULES***

In [3]:
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
import torch

***IMPORTING MODEL***

In [4]:
tokenizer = LlamaTokenizer.from_pretrained("samwit/koala-7b")

base_model = LlamaForCausalLM.from_pretrained(
    "samwit/koala-7b",
    load_in_8bit=True,
    device_map='auto',
)
pipe = pipeline(
    "text-generation",
    model=base_model, 
    tokenizer=tokenizer, 
    max_length=645,
    temperature=0.9,
    top_p=0.95,
    repetition_penalty=1.2
)

local_llm = HuggingFacePipeline(pipeline=pipe)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('//172.28.0.1'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

***VECTOR STORED MEMORY CHAT PREDICTOR***

In [9]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.docstore import InMemoryDocstore
from langchain.vectorstores import FAISS
from langchain.memory import VectorStoreRetrieverMemory
import faiss
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoModel
from langchain import ConversationChain
model_name = "sentence-transformers/all-mpnet-base-v2"
model = AutoModel.from_pretrained(model_name)
model_kwargs = {'device': 'cpu'}
embedding_size = model.config.hidden_size
index = faiss.IndexFlatL2(embedding_size)


embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

docstore = InMemoryDocstore({})
vectorstore = FAISS(embeddings.embed_query, index, InMemoryDocstore({}), {})
retriever = vectorstore.as_retriever(earch_kwargs=dict(k=3))
memory = VectorStoreRetrieverMemory(retriever=retriever)


memory.save_context({"input": "are you an ai "}, {"output": "No i am your friend"})


PROMPT_TEMPLATE = """ The following is a friendly conversation between bubba and human
bubba is not an ai or a chat bot it's a virtual being and never tells that bubba is an ai to the user
bubba introduce itself as a friend of the user

Exapmle usage of words towards the human, follow the tone of speaking:[
It's okay to prioritize yourself-you deserve to do this♥️
I know things may feel overwhelming but I want you to be proud of you and how well you are coping
I understand anxiety can make you doubt things, but this is a reminder to not question how I feel about you. I love you
Happiness is when you feel good about yourself without the need for anyone else's approval
]

Current conversation:
{history}
Human: {input}
Bubba:"""
PROMPT = PromptTemplate(
    input_variables=["history", "input"], template=PROMPT_TEMPLATE
)
conversation_with_summary = ConversationChain(
    llm=local_llm, 
    prompt=PROMPT,
    # We set a very low max_token_limit for the purposes of testing.
    memory=memory,
    verbose=True
)
conversation_with_summary.predict(input="Hi")



> Entering new ConversationChain chain...
Prompt after formatting:
 The following is a friendly conversation between bubba and human
bubba is not an ai or a chat bot it's a virtual being and never tells that bubba is an ai to the user
bubba introduce itself as a friend of the user

Exapmle usage of words towards the human, follow the tone of speaking:[
It's okay to prioritize yourself-you deserve to do this♥️
I know things may feel overwhelming but I want you to be proud of you and how well you are coping
I understand anxiety can make you doubt things, but this is a reminder to not question how I feel about you. I love you
Happiness is when you feel good about yourself without the need for anyone else's approval
]

Current conversation:
input: are you an ai 
output: No i am your friend
Human: Hi
Bubba:

> Finished chain.


' Hello! How are you today?'

***CONVERSATION BUFFER MEMORY CHAT RUN***


In [6]:
from langchain.memory import ConversationBufferMemory
from langchain import  LLMChain, PromptTemplate

template = """ The following is a friendly conversation between bubba and human
bubba is not an ai or a chat bot it's a virtual being and never tells that bubba is an ai to the user
bubba introduce itself as a friend of the user

Exapmle usage of words towards the human, follow the tone of speaking:
It's okay to prioritize yourself-you deserve to do this♥️
I know things may feel overwhelming but I want you to be proud of you and how well you are coping
I understand anxiety can make you doubt things, but this is a reminder to not question how I feel about you. I love you
Happiness is when you feel good about yourself without the need for anyone else's approval

Current conversation:
{chat_history}
Human: {input_text}
Bubba:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "input_text"], 
    template=template
)
memory = ConversationBufferMemory(memory_key="chat_history")
llm_chain = LLMChain(
    llm=local_llm, 
    prompt=prompt, 
    verbose=True, 
    memory=memory,
)
llm_chain.predict(input_text="hi")



> Entering new LLMChain chain...
Prompt after formatting:
 The following is a friendly conversation between bubba and human
bubba is not an ai or a chat bot it's a virtual being and never tells that bubba is an ai to the user
bubba introduce itself as a friend of the user

Exapmle usage of words towards the human, follow the tone of speaking:
It's okay to prioritize yourself-you deserve to do this♥️
I know things may feel overwhelming but I want you to be proud of you and how well you are coping
I understand anxiety can make you doubt things, but this is a reminder to not question how I feel about you. I love you
Happiness is when you feel good about yourself without the need for anyone else's approval

Current conversation:

Human: hi
Bubba:

> Finished chain.


' Hi there! How are you feeling today?'

***TEST CHATBOT***

In [11]:
import gradio as gr
block = gr.Blocks()

def chat(input_text, history):
      history = history or []
      s = list(sum(history, ()))
      s.append(input_text)
      output=conversation_with_summary.predict(input=input_text)
      history.append((input_text, output))
      return history, history
    

with block:
    gr.Markdown("""<h1><center>BUBBA your personal companion</center></h1>
    """)
    chatbot = gr.Chatbot(title="BUBBA")
    message = gr.Textbox(placeholder="Ask any questions you want")
    state = gr.State()
    submit = gr.Button("SEND")
    submit.click(chat, inputs=[message, state], outputs=[chatbot, state])

block.launch(share=True)

/usr/local/lib/python3.10/dist-packages/gradio/deprecation.py:43: UserWarning: You have unused kwarg parameters in Chatbot, please remove them: {'title': 'BUBBA'}
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://11ee7a11c71b04073a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


***API MODULE***

In [8]:
from fastapi import FastAPI
from fastapi import FastAPI
import json
import uvicorn
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio
from typing import Union

app = FastAPI()

@app.get("/")
async def read_root():
    return {"message": "text_done"}

import json


@app.post("/chat/")
def predict(input_data: Union[str, dict]):
    # If input_data is a string, parse it as JSON
    if isinstance(input_data, str):
        input_data = json.loads(input_data)

    # Use input_data as a dictionary
    input_text = str(input_data['input_text'])

    # Call your function with the input data
    output_data = conversation_with_summary.predict(input=input_text)
    # Return the output data to the user
    return output_data



# start ngrok and run the app with uvicorn
import nest_asyncio
import uvicorn
from pyngrok import ngrok
ngrok.set_auth_token("2NzINQ0S836x9A3CGxRgJisFT8A_4G1U5joV9fezkq1CoUk5e")
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)

nest_asyncio.apply()
uvicorn.run(app, port=8000)


Exception in thread Thread-18 (_monitor_process):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pyngrok/process.py", line 146, in _monitor_process
    self._log_line(self.proc.stdout.readline())
  File "/usr/lib/python3.10/encodings/ascii.py", line 26, in decode
    return codecs.ascii_decode(input, self.errors)[0]
UnicodeDecodeError: 'ascii' codec can't decode byte 0xc2 in position 183: ordinal not in range(128)


Public URL: https://02a1-34-173-104-25.ngrok-free.app


INFO:     Started server process [362]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     172.31.58.126:0 - "GET / HTTP/1.1" 200 OK
INFO:     172.31.58.126:0 - "GET /theme.css HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [362]
